# Spark Connectivity Test to Vast DB and Vast S3

## Load Endpoint Environment Variables

These environment variables have been set when your docker container was created.

In [1]:
import os

DOCKER_HOST_OR_IP = os.getenv("DOCKER_HOST_OR_IP")

VASTDB_ENDPOINT = os.getenv("VASTDB_ENDPOINT")
VASTDB_ACCESS_KEY = os.getenv("VASTDB_ACCESS_KEY")
VASTDB_SECRET_KEY = os.getenv("VASTDB_SECRET_KEY")

VASTDB_TWITTER_INGEST_BUCKET = os.getenv("VASTDB_TWITTER_INGEST_BUCKET")
VASTDB_TWITTER_INGEST_SCHEMA = os.getenv("VASTDB_TWITTER_INGEST_SCHEMA")
VASTDB_TWITTER_INGEST_TABLE = os.getenv("VASTDB_TWITTER_INGEST_TABLE")

S3_ENDPOINT = os.getenv("S3A_ENDPOINT")
S3_ACCESS_KEY = os.getenv("S3A_ACCESS_KEY")
S3_SECRET_KEY = os.getenv("S3A_SECRET_KEY")

S3A_ICEBERG_URI = os.getenv("S3A_ICEBERG_URI")

print(f"""
---
DOCKER_HOST_OR_IP={DOCKER_HOST_OR_IP}
---
VASTDB_ENDPOINT={VASTDB_ENDPOINT}
VASTDB_ACCESS_KEY={VASTDB_ACCESS_KEY[-4:]}
VASTDB_SECRET_KEY=****{VASTDB_SECRET_KEY[-4:]}
VASTDB_TWITTER_INGEST_BUCKET={VASTDB_TWITTER_INGEST_BUCKET}
VASTDB_TWITTER_INGEST_SCHEMA={VASTDB_TWITTER_INGEST_SCHEMA}
VASTDB_TWITTER_INGEST_TABLE={VASTDB_TWITTER_INGEST_TABLE}
---
S3_ENDPOINT={S3_ENDPOINT}
S3_ACCESS_KEY={S3_ACCESS_KEY[-4:]}
S3_SECRET_KEY=****{VASTDB_SECRET_KEY[-4:]}
S3A_ICEBERG_URI={S3A_ICEBERG_URI}
---
""")


---
DOCKER_HOST_OR_IP=se-var-vastdb-ingest
---
VASTDB_ENDPOINT=http://172.200.201.1:80
VASTDB_ACCESS_KEY=DYF8
VASTDB_SECRET_KEY=****Usuu
VASTDB_TWITTER_INGEST_BUCKET=csnowdb
VASTDB_TWITTER_INGEST_SCHEMA=social_media
VASTDB_TWITTER_INGEST_TABLE=tweets
---
S3_ENDPOINT=http://172.200.201.1:80
S3_ACCESS_KEY=DYF8
S3_SECRET_KEY=****Usuu
S3A_ICEBERG_URI=s3a://csnow-bucket/iceberg/
---



## Specify other Environment Variables

In [2]:
SPARK_APPLICATION_NAME='Spark Demo'

## Start Spark Session

In [3]:
import socket
import os
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
pd.set_option("max_colwidth", 150)

conf = SparkConf()
conf.setAll([
    ("spark.driver.host", socket.gethostbyname(socket.gethostname())),
    ("spark.sql.execution.arrow.pyspark.enabled", "false"),
     # VASTDB
    ("spark.sql.catalog.ndb", 'spark.sql.catalog.ndb.VastCatalog'),
    ("spark.ndb.endpoint", VASTDB_ENDPOINT),
    ("spark.ndb.data_endpoints", VASTDB_ENDPOINT),
    ("spark.ndb.access_key_id", VASTDB_ACCESS_KEY),
    ("spark.ndb.secret_access_key", VASTDB_SECRET_KEY),
    ("spark.driver.extraClassPath", '/usr/local/spark/jars/spark3-vast-3.4.1-f93839bfa38a/*'),
    ("spark.executor.extraClassPath", '/usr/local/spark/jars/spark3-vast-3.4.1-f93839bfa38a/*'),
    ("spark.sql.extensions", 'ndb.NDBSparkSessionExtension'),
    # ICEBERG
    ("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog"),
    ("spark.sql.catalog.iceberg.type", "hive"),
    ("spark.sql.catalog.iceberg.uri", f"thrift://{DOCKER_HOST_OR_IP}:9083"),
    # S3A
    ("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem"),
    ("fs.s3a.endpoint", S3_ENDPOINT),
    ("fs.s3a.access.key", S3_ACCESS_KEY),
    ("fs.s3a.secret.key", S3_SECRET_KEY),
    ("fs.s3a.endpoint.region", "vast"),
    ("fs.s3a.connection.ssl.enabled", "false"),
])

spark = SparkSession.builder \
    .master("local") \
    .appName(SPARK_APPLICATION_NAME) \
    .config(conf=conf) \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("DEBUG")

print("Spark successfully loaded")

Spark successfully loaded


In [4]:
sc

<SparkContext master=local appName=Spark Demo>

## Connect to Vast DB

### Specify Environment

In [5]:
BUCKET_NAME = VASTDB_TWITTER_INGEST_BUCKET
DATABASE_SCHEMA = VASTDB_TWITTER_INGEST_SCHEMA
TABLE_NAME = VASTDB_TWITTER_INGEST_TABLE

### Connect and run a query

In [6]:
DATABASE_FULLNAME = f"ndb.`{BUCKET_NAME}`.`{DATABASE_SCHEMA}`"

spark.sql(f"create schema if not exists {DATABASE_FULLNAME}")

# Set the database name so we don't have to fully qualify all object names
# https://spark.apache.org/docs/latest/sql-ref-syntax-ddl-usedb.html
# spark.sql(f"use {DATABASE_FULLNAME}")

print(f"Using {DATABASE_FULLNAME=}")

Using DATABASE_FULLNAME='ndb.`csnowdb`.`social_media`'


In [7]:
df = spark.sql(f"""
SELECT 
  *
FROM 
  {DATABASE_FULLNAME}.{TABLE_NAME}
WHERE
  created_at in (
    SELECT 
      MAX(created_at) latest_created_at
    FROM 
      {DATABASE_FULLNAME}.{TABLE_NAME}
  )
ORDER BY id
LIMIT 1
""")

In [8]:
df.toPandas()

,created_at,id,id_str,text
0,1731502432360,-6583528571490526214,-6583528571490526214,prepared for how interesting IoT is!


## Connect to Iceberg on Vast S3

### Specify Environment

In [9]:
spark.sql("SHOW DATABASES in iceberg").toPandas()

,namespace
0,default
1,social_media


In [10]:
spark.sql("SHOW TABLES in iceberg.social_media").toPandas()

,namespace,tableName,isTemporary
0,social_media,twitter_data,False


In [11]:
spark.sql("DESCRIBE EXTENDED iceberg.social_media.twitter_data").toPandas()

,col_name,data_type,comment
0,created_at,bigint,None
1,id,bigint,None
2,id_str,string,None
3,text,string,None
4,,,
5,# Metadata Columns,,
6,_spec_id,int,
7,_partition,struct<>,
8,_file,string,
9,_pos,bigint,


In [12]:
result_df = spark.sql("""
SELECT 
    * 
FROM 
    iceberg.social_media.twitter_data
""")
result_df.toPandas()

,created_at,id,id_str,text
0,1731484717161,-1097640068383552815,-1097640068383552815,blown away by how lovely Security is!
1,1731484717161,2150149765540927710,2150149765540927710,blown away by how smart SoftwareDevelopment is!
2,1731484717161,-3482516525599830825,-3482516525599830825,ready to see how great DataScience is!
3,1731484717161,-4997270603466746885,-4997270603466746885,prepared for how amazing Cybersecurity is!
4,1731484717161,1634671975161576198,1634671975161576198,finally got how fantastic Agile is!
...,...,...,...,...
95,1731484738818,5943826959473537337,5943826959473537337,eager to see how perfect Security is!
96,1731484738818,-5823557175065379066,-5823557175065379066,amazed by how incredible IoT is!
97,1731484738818,1089710928912076635,1089710928912076635,prepared for how incredible OpenSource is!
98,1731484738818,3106624588108967535,3106624588108967535,ready to see how fantastic SecurityEngineering is!
